In [68]:
import re
import pandas as pd
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
kor_corpus = pd.read_csv("data/kor_news.csv", encoding="cp949")
yonhap = kor_corpus["contents"]
yonhap

0       (광주=연합뉴스) 여운창 기자 = 날로 커가는 피부·미용분야의 에스테틱 시장을 개척...
1       (구미=연합뉴스) 박순기 기자 = 노래·춤 경연으로 청소년 꿈과 열정을 키워주는 '...
2       (부산=연합뉴스) 박창수 기자 = 고신대 복음병원은 카자흐스탄 제2 도시 알마티에 ...
3        (대구=연합뉴스) 한국패션문화산업진흥원은 2016 대구국제패션문화페스티벌 자원봉사...
4       (세종=연합뉴스) 김영만 기자 = 한국의 고고도 미사일 방어체계(THAAD·사드) ...
                              ...                        
2585    (부산=연합뉴스) 신정훈 기자 = 부산시가 글로벌 명품시장으로 선정된 국제시장을 비...
2586    (서울=연합뉴스) 김중배 기자 = 김종덕 문화체육관광부 장관은 24일 국립현대미술관...
2587    (서울=연합뉴스) 이영재 기자 = 한반도 긴장 해소를 위해 열린 남북 고위급접촉의 ...
2588    (서울=연합뉴스) 김중배 기자 = 김종덕 문화체육관광부 장관이 지난 21일로 취임 ...
2589    (서울=연합뉴스) 현혜란 기자 = "파리 센강 옆은 건물이라도 멋있지. 한강은 주변...
Name: contents, Length: 2590, dtype: object

In [69]:
okt = Okt()
# total_news = list()
# for news in kor_corpus["contents"].head():
#     pos_news = [','.join(t[:-1]) for t in okt.pos(news) if (t[1] == "Noun")]
#     total_news.append(' '.join(pos_news))

total_news = []
stopword = ['것', '수', '저']
for news in yonhap[:1000]:
    pos_news = re.sub('.* 기자 = ', '', news)
    pos_news = [','.join(t[:-1]) for t in okt.pos(pos_news) if ((t[1]=='Noun') & (t[0] not in stopword))]
    total_news.append(' '.join(pos_news))

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

In [70]:
kor_vectorizer = CountVectorizer()
kor_bow = kor_vectorizer.fit_transform(total_news)
kor_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [51]:
from sklearn.feature_extraction.text import TfidfTransformer

In [71]:
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(kor_bow.toarray())
tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [63]:
def find_most_similar_news_idf(index, tfidf, corpus):
    idx = (-cosine_similarity(tfidf[index], tfidf)[0]).argsort()[1]
    
    return corpus[idx]

In [76]:
idx = 300
print(yonhap[300])

most_similar = find_most_similar_news_idf(idx, tfidf, yonhap)
print("\n", most_similar)

(서울=연합뉴스) 곽명일 기자 = 북한당국이 주민을 상대로 자본주의 문화에 대한 단속과 통제를 강화하고 있지만, '태양의 후예'와 같은 한국 드라마를 몰래 시청하는 북한 주민들이 많다고 북한 전문매체인 데일리NK가 4일 보도했다. 평안남도 소식통은 이 매체에 "최근 젊은 청년들 속에서 '태양의 후예'라는 한국 드라마가 인기를 끌면서 날이 새는 줄도 모르고 시청하고 있다"면서 "이 드라마에 대한 소문이 퍼지자 어른·아이 할 것 없이 너도나도 (드라마를) 보기 위해 애쓰고 있는 상황"이라고 전했다. 소식통은 이어 "시장에서 상인들에게 다가가 '아랫동네'(한국)것이 없느냐'고 슬쩍 말을 건네면 대뜸 '태양의 후예'를 소개한다"며 "새 드라마 원본(CD)은 5만 원(한국 돈 7천 원)에 판매되고, 복사본은 그 절반 값에 거래된다"고 설명했다. 전체기사 본문배너 그러면서 "(주민들은)'200일 전투'의 쌓인 피로를 한국 드라마를 시청하거나 가요를 감상하면서 해소한다"며 "한류의 확산으로 볼거리가 없는 조선중앙TV를 외면하는 주민이 늘어나고 있다"고 덧붙였다. 앞서 이 매체는 북한당국이 중국산 노트텔(EVD 플레이어)을 남한 드라마 시청의 주요 수단으로 지목하면서 수입을 전면 금지하는 지시를 내렸다고 지난달 23일 보도한 바 있다.

 (하노이=연합뉴스) 김문성 특파원 = 배정호 민주평화통일자문회의 사무처장은 27일 "북한의 전통적인 우방국이자 여전히 정치, 경제, 군사적 교류를 이어가는 베트남의 대북제재 이행은 북한의 태도변화에 큰 전환점이 될 수 있다"며 베트남의 적극적인 제재 동참을 촉구했다. 배 사무처장은 이날 베트남 하노이 롯데호텔에서 민주평통 주최로 열린 '2016 한·베트남 평화통일포럼'에서 기조연설을 통해 "유엔 제재가 빈틈없이 이행되도록 한국과 베트남이 적극적으로 협력하는 것이 무엇보다 필요하다"고 강조했다. 그는 "2014년 북한과 베트남이 500만 달러(57억 원)의 투자와 교역을 진행했고 베트남에 북한 식당도 운영되고 있다"며 북한의 돈줄을 죄고 비